<a href="https://colab.research.google.com/github/hypo69/hypotez/blob/master/SANDBOX/davidka/LLM-HOWTO/LLM_%D1%88%D0%B0%D0%B3_%D0%BF%D0%B5%D1%80%D0%B2%D1%8B%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создание сети



**Важно:** Эта сеть будет **очень простой** (один скрытый слой, ручная реализация без библиотек типа TensorFlow/PyTorch) и **не является LLM (Large Language Model)**.  Эта сеть для иллюстрации базовых принципов нейронных сетей (нейроны, слои, веса, обучение через обратное распространение ошибки).


**Объяснение кода:**

1.  **Классы (`Input`, `Neuron`, `Layer`, `NeuroNetwork`):** Структура повторяет статью. `Input` хранит вес и ссылку на предыдущий нейрон. `Neuron` содержит значение, список входов, ошибку и дельту для обучения, а также методы для расчета значения. `Layer` группирует нейроны. `NeuroNetwork` объединяет слои, управляет обучением и тестированием.
2.  **Инициализация (`__init__`):** В конструкторе `NeuroNetwork` создаются слои нужного размера. Размеры скрытых слоев рассчитываются по формуле из статьи. Веса инициализируются случайно (`random.random()`).
3.  **Функции активации (`sigmoid`, `sigmoid_derivative`):** Используется сигмоидальная функция и ее производная, как описано. Добавлена защита от математического переполнения в `sigmoid`.
4.  **Прямой проход (`forward_pass`, `get_prediction`):** Данные проходят от входа к выходу. Каждый нейрон (кроме входных) вычисляет свою взвешенную сумму входов (`get_input_sum`) и применяет функцию активации (`calculate_value`). `get_prediction` запускает `forward_pass` и возвращает значения выходных нейронов.
5.  **Обратное распространение ошибки (`backpropagate`):** Это ядро обучения.
    *   Сначала вычисляется ошибка и "дельта" для выходного слоя (насколько сильно выходной нейрон ошибся и как сильно его входная сумма влияет на ошибку).
    *   Затем ошибка "распространяется" назад по слоям. Ошибка каждого нейрона скрытого слоя зависит от ошибок нейронов следующего слоя (справа) и весов связей между ними. Рассчитывается дельта для каждого скрытого нейрона.
    *   Наконец, зная дельты всех нейронов, веса всех связей корректируются (`w = w + learning_rate * delta_нейрона_справа * значение_нейрона_слева`) так, чтобы уменьшить общую ошибку сети.
6.  **Обучение (`train`, `train_once`):** `train` управляет процессом обучения в течение заданного числа `epochs`. В каждой эпохе она проходит по всему `dataset`. `train_once` обрабатывает один пример: выполняет прямой проход, затем обратное распространение для коррекции весов. Датасет перемешивается в каждой эпохе для лучшего обучения.
7.  **Тестирование (`test`):** Подает на вход сети тестовые данные, выполняет прямой проход и выводит предсказанный результат. Для задач классификации (как "ИЛИ") результат часто округляют до 0 или 1.
8.  **Сохранение/Загрузка (`save_model`, `load_model`):** Используется стандартная библиотека Python `pickle`. `save_model` сериализует весь объект `NeuroNetwork` (включая все слои, нейроны и их веса) в файл. `load_model` десериализует объект из файла, позволяя восстановить обученную сеть.


**Следующие шаги (Куда двигаться дальше):**

*   **Поэкспериментировать:** Изменить скорость обучения (`learning_rate`), количество скрытых нейронов, количество эпох, попробовать другую функцию активации (например, ReLU).
*   **Другие задачи:** Попробовать обучить сеть на других логических операциях (AND, XOR - XOR сложнее и может потребовать более глубокой сети или больше нейронов).
*   **Библиотеки:** Изучить библиотеки `NumPy` (для быстрых операций с матрицами), `TensorFlow` или `PyTorch`. Они предоставляют готовые слои, функции активации, оптимизаторы и используют GPU для ускорения вычислений, что необходимо для настоящих LLM.
Эта простая сеть **НЕ** LLM. LLM используют архитектуру Transformer, огромные датасеты и триллионы операций для обучения.

## *Код*

In [ ]:
import random
from math import exp, ceil
import pickle # Для сохранения и загрузки модели

# --- Часть 1: Инициализация ---

# Класс для входа нейрона (хранит ссылку на предыдущий нейрон и вес связи)
class Input:
    def __init__(self, prev_neuron, weight):
        self.prev_neuron = prev_neuron # Нейрон из предыдущего слоя
        self.weight = weight         # Вес связи

# Класс Нейрона
class Neuron:
    def __init__(self, layer, previous_layer):
        self._layer = layer # Слой, к которому принадлежит нейрон
        self.value = 0.0    # Текущее значение/активация нейрона (инициализируем 0)
        self.inputs = []    # Список входящих связей (объекты Input)
        self.error = 0.0    # Ошибка нейрона (для обратного распространения)
        self.delta = 0.0    # Дельта ошибки (ошибка * производная активации) - для расчета градиентов
        self.last_input_sum = 0.0 # Сумма взвешенных входов (нужна для вычисления производной)

        # Создаем входы, если это не входной слой (т.е. есть предыдущий слой)
        if previous_layer:
            self.inputs = [Input(prev_neuron, random.random()) # Случайный вес от 0.0 до 1.0
                           for prev_neuron in previous_layer.neurons]

    # Установить значение нейрона (используется для входного слоя)
    def set_value(self, val):
        self.value = float(val) # Убедимся, что значение - float

    # Проверить, есть ли у нейрона входы (т.е. не является ли он нейроном входного слоя)
    def is_no_inputs(self):
        return not self.inputs

    # Рассчитать взвешенную сумму входов
    def get_input_sum(self):
        # Сумма = Σ (значение_предыдущего_нейрона * вес_связи)
        total_sum = sum(inp.prev_neuron.value * inp.weight for inp in self.inputs)
        return total_sum

    # Рассчитать и обновить значение (активацию) нейрона на основе входов
    def calculate_value(self):
        # Нейроны входного слоя свое значение получают извне (set_value)
        if self.is_no_inputs():
            return

        # 1. Считаем взвешенную сумму входов
        input_sum = self.get_input_sum()
        self.last_input_sum = input_sum # Сохраняем для расчета производной при обучении

        # 2. Применяем функцию активации (сигмоиду)
        self.value = self._layer.network.activate_func(input_sum)

# Класс Слоя
class Layer:
    def __init__(self, layer_size, prev_layer, parent_network):
        self.prev_layer = prev_layer       # Ссылка на предыдущий слой (None для входного)
        self.network = parent_network    # Ссылка на родительскую сеть
        # Создаем нейроны для этого слоя
        self.neurons = [Neuron(self, prev_layer) for _ in range(layer_size)]

    # Установить входные данные для слоя (используется для входного слоя)
    def set_input_data(self, val_list):
        if len(val_list) != len(self.neurons):
            raise ValueError("Размер входных данных не совпадает с количеством нейронов входного слоя")
        for i in range(len(val_list)):
            self.neurons[i].set_value(val_list[i])

# Класс Нейронной Сети
class NeuroNetwork:
    def __init__(self, input_l_size, output_l_size, hidden_layers_count=1, learning_rate=0.1): # Уменьшил learning_rate для стабильности
        self.selected_layer = None # Временный указатель для постройки слоев
        self.l_count = hidden_layers_count + 2 # Общее кол-во слоев (входной + скрытые + выходной)

        # Формула расчета размера скрытых слоев (из статьи)
        # Добавил max(1, ...) чтобы слой не был нулевого размера
        hidden_l_size = max(1, min(input_l_size * 2 - 1, ceil(input_l_size * 2 / 3 + output_l_size)))

        # --- Параметры обучения ---
        self.learning_rate = learning_rate          # Скорость обучения
        self.activate_func = NeuroNetwork.sigmoid   # Функция активации
        self.derivate_func = NeuroNetwork.sigmoid_derivative # Производная функции активации

        # --- Создание слоев ---
        self.layers = []
        for i in range(self.l_count):
            # Определяем размер текущего слоя
            if i == 0: # Входной слой
                current_layer_size = input_l_size
                prev_layer_ref = None
            elif i == self.l_count - 1: # Выходной слой
                current_layer_size = output_l_size
                prev_layer_ref = self.layers[-1] # Последний добавленный слой
            else: # Скрытый слой
                current_layer_size = hidden_l_size
                prev_layer_ref = self.layers[-1] # Последний добавленный слой

            # Создаем слой и добавляем в список
            new_layer = Layer(current_layer_size, prev_layer_ref, self)
            self.layers.append(new_layer)

        print(f"Сеть создана: Входной={input_l_size}, Скрытый={hidden_l_size}x{hidden_layers_count}, Выходной={output_l_size}")

    # --- Статические методы для функций активации ---
    @staticmethod
    def sigmoid(x):
        # Добавим защиту от переполнения для exp()
        if x < -700: return 0.0
        if x > 700: return 1.0
        try:
            return 1 / (1 + exp(-x))
        except OverflowError:
            return 0.0 if x < 0 else 1.0


    @staticmethod
    def sigmoid_derivative(x):
        # Производная сигмоиды: sigmoid(x) * (1 - sigmoid(x))
        sig_x = NeuroNetwork.sigmoid(x)
        return sig_x * (1 - sig_x)

    # --- Часть 2: Обучение и Тестирование ---

    # Установка входных данных для всей сети
    def set_input_data(self, val_list):
        self.layers[0].set_input_data(val_list) # Устанавливаем данные на первый (входной) слой

    # Прямой проход (Forward Pass) - вычисление результата сети
    def forward_pass(self):
        # Проходим по всем слоям, начиная со второго (первого скрытого)
        for i in range(1, self.l_count):
            layer = self.layers[i]
            # Каждый нейрон слоя вычисляет свое значение на основе предыдущего слоя
            for neuron in layer.neurons:
                neuron.calculate_value()

    # Получение предсказания (результата) сети
    def get_prediction(self):
        # Сначала выполняем прямой проход, чтобы обновить все значения
        self.forward_pass()
        # Собираем значения с нейронов выходного слоя
        output_layer = self.layers[-1]
        out_data = [neuron.value for neuron in output_layer.neurons]
        return out_data

    # Обратное распространение ошибки (Backpropagation) и обновление весов для одного примера
    def backpropagate(self, expected_output):
        output_layer = self.layers[-1]

        # Убедимся, что размер ожидаемого выхода совпадает с выходным слоем
        if len(expected_output) != len(output_layer.neurons):
             raise ValueError("Размер ожидаемого выхода не совпадает с размером выходного слоя")

        # 1. Вычисление ошибки и дельты для выходного слоя
        for i, neuron in enumerate(output_layer.neurons):
            error = expected_output[i] - neuron.value
            neuron.error = error
            # Дельта = Ошибка * Производная(взвешенная_сумма_входов_нейрона)
            neuron.delta = error * self.derivate_func(neuron.last_input_sum)

        # 2. Распространение ошибки на скрытые слои (от предпоследнего к первому скрытому)
        for i in range(self.l_count - 2, 0, -1): # Идем от слоя l-2 до слоя 1
            current_layer = self.layers[i]
            next_layer = self.layers[i+1] # Слой справа (ближе к выходу)

            for j, neuron in enumerate(current_layer.neurons):
                # Ошибка нейрона = Сумма (дельта_нейрона_справа * вес_связи_между_ними)
                error_sum = sum(next_neuron.delta * next_neuron.inputs[j].weight
                                for next_neuron in next_layer.neurons)
                neuron.error = error_sum
                # Дельта = Ошибка * Производная(взвешенная_сумма_входов_нейрона)
                neuron.delta = error_sum * self.derivate_func(neuron.last_input_sum)

        # 3. Обновление весов во всех слоях (начиная с первого скрытого)
        for i in range(1, self.l_count): # Идем от слоя 1 до слоя l-1
            current_layer = self.layers[i]
            for neuron in current_layer.neurons:
                for input_connection in neuron.inputs:
                    # Обновление веса: w = w + learning_rate * дельта_нейрона * значение_нейрона_слева
                    gradient = neuron.delta * input_connection.prev_neuron.value
                    input_connection.weight += self.learning_rate * gradient

    # Обучение на одном примере данных (прямой проход + обратное распространение)
    def train_once(self, input_data, expected_output):
        # 1. Устанавливаем входные данные
        self.set_input_data(input_data)
        # 2. Выполняем прямой проход (чтобы получить текущий выход и last_input_sum)
        self.forward_pass()
        # 3. Выполняем обратное распространение ошибки и обновляем веса
        self.backpropagate(expected_output)
        # Можно вернуть ошибку для мониторинга (например, среднеквадратичную)
        output_layer = self.layers[-1]
        mse = sum((expected_output[i] - output_layer.neurons[i].value)**2 for i in range(len(expected_output))) / len(expected_output)
        return mse


    # Обучение на всем наборе данных в течение нескольких эпох (итераций)
    def train(self, dataset, epochs=1000):
        print(f'\nОБУЧЕНИЕ НАЧАТО ({epochs} эпох)...')
        for epoch in range(epochs):
            total_epoch_error = 0
            # Перемешиваем датасет в начале каждой эпохи (для стохастичности)
            random.shuffle(dataset)
            for case in dataset:
                input_data = case[0]
                # Ожидаемый результат должен быть списком/кортежем
                expected_res = case[1] if isinstance(case[1], (list, tuple)) else [case[1]]
                case_error = self.train_once(input_data, expected_res)
                total_epoch_error += case_error

            # Выводим среднюю ошибку за эпоху (каждые N эпох для краткости)
            if (epoch + 1) % 100 == 0 or epoch == 0:
                avg_error = total_epoch_error / len(dataset)
                print(f'Эпоха {epoch+1}/{epochs}, Средняя ошибка (MSE): {avg_error:.6f}')

        print(f'\nОБУЧЕНИЕ ЗАВЕРШЕНО!\n')

    # Тестирование сети на новых данных
    def test(self, data, op_name="OPERATION"):
        print('\nТЕСТИРОВАНИЕ ДАННЫХ:')
        for case in data:
            input_values = case # Входные данные для теста
            self.set_input_data(input_values)
            prediction = self.get_prediction() # Получаем предсказание
            # Округляем результат для задач классификации (как OR)
            rounded_res = [round(p) for p in prediction]
            # Форматируем вывод
            input_str = ", ".join(map(str, input_values))
            pred_str = ", ".join(map(lambda x: f"{x:.4f}", prediction))
            rounded_str = ", ".join(map(str, rounded_res))
            print(f'{input_str} -> Предсказано: [{pred_str}] (Округленно: [{rounded_str}])')

    # --- Сохранение и Загрузка Модели ---

    # Сохранение обученной модели в файл
    def save_model(self, filename="neuro_network_model.pkl"):
        try:
            with open(filename, 'wb') as f:
                pickle.dump(self, f)
            print(f"Модель успешно сохранена в файл: {filename}")
        except Exception as e:
            print(f"Ошибка при сохранении модели: {e}")

    # Загрузка модели из файла (статический метод, т.к. вызывается до создания объекта)
    @staticmethod
    def load_model(filename="neuro_network_model.pkl"):
        try:
            with open(filename, 'rb') as f:
                model = pickle.load(f)
            print(f"Модель успешно загружена из файла: {filename}")
            return model
        except FileNotFoundError:
            print(f"Ошибка: Файл модели не найден: {filename}")
            return None
        except Exception as e:
            print(f"Ошибка при загрузке модели: {e}")
            return None



## Подключение диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/')  # Просто подключаем весь диск

Mounted at /content/drive


## Загрузка датсета

## Запуск обучения

In [ ]:
# 1. Создание сети (2 входа, 1 выход для операции OR)


# Попробуем скорость обучения поменьше (0.1) и больше эпох
nn = NeuroNetwork(input_l_size=2, output_l_size=1, hidden_layers_count=1, learning_rate=0.1)

# 2. Подготовка обучающего набора данных для операции "ИЛИ" (OR)
# Формат: [[входные_данные], ожидаемый_выход]
dataset_or = [
    [[0, 0], [0]],
    [[0, 1], [1]],
    [[1, 0], [1]],
    [[1, 1], [1]]
]

# 3. Обучение сети
nn.train(dataset_or, epochs=10000) # Увеличим количество эпох

# 4. Тестирование сети
test_data_or = [[0, 0], [0, 1], [1, 0], [1, 1]]
nn.test(test_data_or, 'OR')

# 5. Сохранение обученной модели
model_filename = "my_or_network.pkl"
nn.save_model(model_filename)

# --- Пример загрузки и использования сохраненной модели ---
print("\n--- Загрузка и тест сохраненной модели ---")
loaded_nn = NeuroNetwork.load_model(model_filename)

if loaded_nn:
    # Тестируем загруженную модель
    loaded_nn.test(test_data_or, 'OR (loaded)')

    # Можно даже дообучить загруженную модель, если нужно
    # print("\nДообучение загруженной модели...")
    # loaded_nn.train(dataset_or, epochs=1000)
    # loaded_nn.test(test_data_or, 'OR (retrained)')
else:
    print("Не удалось загрузить модель для теста.")